# Capstone Project | Shreya Gupta

## Introduction

I am a graduate student who is looking to move to DC and trying to identify which are the best neighbourhoods from a location perspective. In order to identify 'happening' neighbourhoods, I will use number of restaurants in the neighbourhood as a proxy. I will then identify neighbourhoods with a high density of restaurants. This analysis will aid to finally identifying the perfect area for me to live.  

## Data

Foursquare data from thier API, will be used to collect venue category and location data for the Washington, D.C. area. 

Once the data is in a dataframe, I will run a k-means cluster on a venue by venue basis, latitude, longtitude and ultimately determine key clusters. I'll then map these clusters onto a map of Washington D.C. using the folium library for visualization of these neighborhoods.

## Methodology

In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [4]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [11]:
import requests
import random

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

from IPython.display import Image 
from IPython.core.display import HTML 
    
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                       

In [6]:
url = 'https://maps2.dcgis.dc.gov/dcgis/rest/services/DCGIS_DATA/Administrative_Other_Boundaries_WebMercator/MapServer/17/query?where=1%3D1&outFields=NBH_NAMES&outSR=4326&f=json'
results = requests.get(url).json()
results   

{'displayFieldName': 'NAME',
 'fieldAliases': {'NBH_NAMES': 'NEIGHBORHOOD NAMES'},
 'geometryType': 'esriGeometryPolygon',
 'spatialReference': {'wkid': 4326, 'latestWkid': 4326},
 'fields': [{'name': 'NBH_NAMES',
   'type': 'esriFieldTypeString',
   'alias': 'NEIGHBORHOOD NAMES',
   'length': 254}],
 'features': [{'attributes': {'NBH_NAMES': 'Congress Heights, Bellevue, Washington Highlands'},
   'geometry': {'rings': [[[-76.99401890037231, 38.84519662346874],
      [-76.99351265380169, 38.844912830429294],
      [-76.9923120753984, 38.844159741588356],
      [-76.99231497127192, 38.84402371311473],
      [-76.98990844795414, 38.84443296491636],
      [-76.98971319250782, 38.84444510927833],
      [-76.98931784704706, 38.844502367491366],
      [-76.98850917322143, 38.84464030121506],
      [-76.98850813078589, 38.843244259906825],
      [-76.98850754541213, 38.84186245505666],
      [-76.9885076423583, 38.83983762553577],
      [-76.9882849731025, 38.83784385125863],
      [-76.98811

In [12]:
# function that extracts the category of the venue
def get_neighborhoods(row):
    neighborhoods_list = row['attributes']        
    if len(neighborhoods_list) == 0:
        return None
    else:
        return neighborhoods_list[0]['NBH_NAMES']

In [13]:
locations = results['features']
    
neighborhood_locations = json_normalize(locations) # flatten JSON

# filter columns
filter_columns = ['attributes.NBH_NAMES', 'geometry.rings']
neighborhood_locations = neighborhood_locations.loc[:, filter_columns]

# filter the category for each row
#neighborhood_locations['attributes.NBH_NAMES'] = neighborhood_locations.apply(get_neighborhoods, axis=1)

# clean columns
neighborhood_locations.columns = [col.split(".")[-1] for col in neighborhood_locations.columns]

# define the dataframe columns
column_names = ['Neighborhoods', 'Location'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

#neighborhood_locations[['team1','team2']] = pd.DataFrame(df2.rings.tolist(), index= df2.index)

neighborhoods['Neighborhoods'] = neighborhood_locations['NBH_NAMES']
neighborhoods['Location'] = neighborhood_locations['rings'].str[0].str[0]
neighborhoods['Latitude'] = neighborhoods['Location'].str[1]
neighborhoods['Longitude'] = neighborhoods['Location'].str[0]
neighborhoods.drop(columns = 'Location', inplace = True)

neighborhoods

,Neighborhoods,Latitude,Longitude
0,"Congress Heights, Bellevue, Washington Highlands",38.845197,-76.994019
1,"Douglas, Shipley Terrace",38.852871,-76.974718
2,"Woodland/Fort Stanton, Garfield Heights, Knox ...",38.860672,-76.968773
3,"Near Southeast, Navy Yard",38.877118,-76.987260
4,"River Terrace, Benning, Greenway, Dupont Park",38.889960,-76.937601
5,"Downtown, Chinatown, Penn Quarters, Mount Vern...",38.905657,-77.009056
6,"West End, Foggy Bottom, GWU",38.908985,-77.050111
7,"Mayfair, Hillbrook, Mahaning Heights",38.905974,-76.947547
8,"Deanwood, Burrville, Grant Park, Lincoln Heigh...",38.889764,-76.913220
9,"Shaw, Logan Circle",38.914101,-77.031954


In [30]:
CLIENT_ID = 'DHKFSOIOAUXAVXG5EI1UCVR3X2NHJK3UZNMHRJ1IPGNXS4FY'
CLIENT_SECRET = 'DO5AQAYQG4NEKIABT3B2BH0NAYDRK5DAVSAGZZFZUC5VMFQS'
VERSION = '20180604'
LONGITUDE = -77.036561
LATITUDE = 38.902927
RADIUS = 500
LIMIT = 100

{'meta': {'code': 200, 'requestId': '5ee548d37ab1524df81133a9'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Connecticut Avenue - K Street',
  'headerFullLocation': 'Connecticut Avenue - K Street, Washington',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 72,
  'suggestedBounds': {'ne': {'lat': 38.9074270045, 'lng': -77.03078929786915},
   'sw': {'lat': 38.898426995499996, 'lng': -77.04233270213086}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4a5e18c0f964a52006be1fe3',
       'name': 'The St. Regis Washington, D.C.',
       'location': {'address': '923 16th St NW',
        'crossStreet': 'at K St NW',
        'lat

In [40]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            RADIUS,
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhoods', 
                  'Neighborhoods Latitude', 
                  'Neighborhoods Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [41]:
DC_venues = getNearbyVenues(names=neighborhoods['Neighborhoods'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude'])


DC_venues

Congress Heights, Bellevue, Washington Highlands
Douglas, Shipley Terrace
Woodland/Fort Stanton, Garfield Heights, Knox Hill
Near Southeast, Navy Yard
River Terrace, Benning, Greenway, Dupont Park
Downtown, Chinatown, Penn Quarters, Mount Vernon Square, North Capitol Street
West End, Foggy Bottom, GWU
Mayfair, Hillbrook, Mahaning Heights
Deanwood, Burrville, Grant Park, Lincoln Heights, Fairmont Heights
Shaw, Logan Circle
Eastland Gardens, Kenilworth
Ivy City, Arboretum, Trinidad, Carver Langston
Georgetown, Burleith/Hillandale
Dupont Circle, Connecticut Avenue/K Street
Howard University, Le Droit Park, Cardozo/Shaw
Southwest Employment Area, Southwest/Waterfront, Fort McNair, Buzzard Point
Capitol View, Marshall Heights, Benning Heights
Spring Valley, Palisades, Wesley Heights, Foxhall Crescent, Foxhall Village, Georgetown Reservoir
North Michigan Park, Michigan Park, University Heights
North Cleveland Park, Forest Hills, Van Ness
Brightwood Park, Crestwood, Petworth
Friendship Height

,Neighborhoods,Neighborhoods Latitude,Neighborhoods Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Congress Heights, Bellevue, Washington Highlands",38.845197,-76.994019,Entertainment & Sports Arena,38.846825,-76.991215,Basketball Stadium
1,"Congress Heights, Bellevue, Washington Highlands",38.845197,-76.994019,Popeyes Louisiana Kitchen,38.844926,-76.997012,Fried Chicken Joint
2,"Congress Heights, Bellevue, Washington Highlands",38.845197,-76.994019,China Inn Carryout,38.845672,-76.996879,Fast Food Restaurant
3,"Congress Heights, Bellevue, Washington Highlands",38.845197,-76.994019,Malcolm X and MLK,38.843144,-76.998744,Road
4,"Douglas, Shipley Terrace",38.852871,-76.974718,Anacostia Community Museum,38.856728,-76.976899,Museum
5,"Douglas, Shipley Terrace",38.852871,-76.974718,Downtown Chicken,38.850802,-76.974745,Wings Joint
6,"Douglas, Shipley Terrace",38.852871,-76.974718,Douglass Community Recreation Center,38.852218,-76.977411,Park
7,"Douglas, Shipley Terrace",38.852871,-76.974718,Anacostia Art Gallery & Boutique,38.856265,-76.975281,Art Gallery
8,"Douglas, Shipley Terrace",38.852871,-76.974718,Wamata Bus Stop #1003035,38.853741,-76.970322,Bus Stop
9,"Douglas, Shipley Terrace",38.852871,-76.974718,Alabama Convenience,38.849809,-76.972213,Convenience Store


,Neighborhoods,Neighborhoods Latitude,Neighborhoods Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Woodland/Fort Stanton, Garfield Heights, Knox ...",38.860672,-76.968773,Alabama Ave-Naylor Rd-Good Hope Rd Crossroads,38.860289,-76.968404,Intersection
1,"Woodland/Fort Stanton, Garfield Heights, Knox ...",38.860672,-76.968773,Murrys,38.861039,-76.967741,Grocery Store
2,"River Terrace, Benning, Greenway, Dupont Park",38.889960,-76.937601,Denny's,38.890438,-76.938325,Breakfast Spot
3,"River Terrace, Benning, Greenway, Dupont Park",38.889960,-76.937601,Globe Liquor,38.889324,-76.936872,Shop & Service
4,"River Terrace, Benning, Greenway, Dupont Park",38.889960,-76.937601,America's Best Wings,38.889456,-76.936816,American Restaurant
5,"River Terrace, Benning, Greenway, Dupont Park",38.889960,-76.937601,Shrimp Boat,38.889843,-76.937757,Seafood Restaurant
6,"River Terrace, Benning, Greenway, Dupont Park",38.889960,-76.937601,Zipcar,38.890487,-76.937436,Parking
7,"River Terrace, Benning, Greenway, Dupont Park",38.889960,-76.937601,Benning Road Metro Station,38.890361,-76.937412,Metro Station
8,"River Terrace, Benning, Greenway, Dupont Park",38.889960,-76.937601,Benning Road & East Capitol,38.889404,-76.937386,Intersection
9,"Downtown, Chinatown, Penn Quarters, Mount Vern...",38.905657,-77.009056,2M ST. Gym and Fitness Center,38.905989,-77.008958,Gym


In [42]:
from sklearn.preprocessing import StandardScaler

In [43]:
from folium import plugins
# create map
LATITUDE = 38.902927
LONGITUDE = -77.036561
map_clusters = folium.Map(location=[LATITUDE, LONGITUDE], tiles = "Stamen Terrain", zoom_start=12)

clusters = plugins.MarkerCluster().add_to(map_clusters)

# add markers to the map
markers_colors = []
for name, cat, lat, lon in zip(DC_venues['Venue'], DC_venues['Venue Category'], DC_venues['Venue Latitude'], DC_venues['Venue Longitude']):
    label = folium.Popup('Name: ' + str(name) + ', Category: ' + str(cat) , parse_html=True)
    folium.Marker(
        [lat, lon],
        popup=label,
        ).add_to(clusters)
       
map_clusters

## Results

From the map above, we can see the different clusters of venues across DC and as we zoom in further, the clusters are displayed at a more granular level. Additionally, the map show the quadrant of area the cluster is refering to. (Please hover over map to see quadrants)

## Discussion

Based on this study, the two areas near Georgetown University that stand out are Logan Cirle, DuPont Area with 68 venues and the Goergetown/Glover Park area with 54. Something in the middle of those two areas would be the idea location to live.

## Conclusion

The conclusion of this study is students should love in the two areas identified above if th